# MovieLens Reccomendation

>This notebook makes use of a simple library called varname, which easily enables the conversion of variable names into strings, if you have not already installed it uncomment the cell below

In [ ]:
#!pip install varname

In [2]:
#Importing necessary libraries
import numpy as np
import pandas as pd
from Code import functions as func
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise.prediction_algorithms import SVD, KNNWithMeans, KNNBasic, KNNBaseline
import numpy as np

In [3]:
#Reading in data
ratings=pd.read_csv('Data/ratings.csv')
links=pd.read_csv('Data/links.csv')
tags=pd.read_csv('Data/tags.csv')
movies=pd.read_csv('Data/movies.csv')
#See functions.py for details on this function
func.get_names(ratings, links, tags, movies)
df_list = [ratings, links, tags, movies]

We have data spearated into multiple dataframes at the moment. Let's review the data quickly to see what features are where.

In [4]:
#displaying all heads
for df in df_list:
    print(df.__dfname__)
    display(df.head(2))

ratings


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247


links


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0


tags


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996


movies


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


Now that we have an idea of what features we have lets combine them into a dataframe

In [28]:
#join the tables together
df_joined= pd.merge(movies, ratings, on='movieId')
df_joined.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


Since we're working on a baseline model, we're going to drop genres for now, though we will come back to that later. We drop title for performance purposes and can map back to the title when the process is finished. We also drop timestamp here because we're not using it as of yet.

In [35]:
df_new = df_joined.drop(['genres', 'title','timestamp'], axis=1)
df_new.head()

,movieId,userId,rating
0,1,1,4.0
1,1,5,4.0
2,1,7,4.5
3,1,15,2.5
4,1,17,4.5


In [5]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['movieId', 'userId', 'rating']],reader)
dataset = data.build_full_trainset()

In [6]:
#need to add more to the grid search later
params = {'n_factors': [20, 50, 100],
         'reg_all': [0.02, 0.05, 0.1]}
g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
g_s_svd.fit(data)

print(g_s_svd.best_score)
print(g_s_svd.best_params)

{'rmse': 0.8742810419408571, 'mae': 0.673350053769575}
{'rmse': {'n_factors': 50, 'reg_all': 0.05}, 'mae': {'n_factors': 50, 'reg_all': 0.05}}


### using KNN


In [7]:
knn_baseline = KNNBaseline(sim_options={'name':'pearson', 'user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [8]:
for i in cv_knn_baseline.items():
    print(i)

np.mean(cv_knn_baseline['test_rmse'])

('test_rmse', array([0.88612437, 0.88580136, 0.88382273, 0.88338085, 0.88371912]))
('test_mae', array([0.68159047, 0.68343112, 0.67985462, 0.6788898 , 0.68130153]))
('fit_time', (31.508434772491455, 31.451374292373657, 35.743053674697876, 32.92459321022034, 30.071699857711792))
('test_time', (12.188941955566406, 12.954964637756348, 13.982067584991455, 12.320340633392334, 11.37741494178772))


0.8845696871826204

In [13]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.260025037657393, details={'was_impossible': False})

In [11]:
df_joined.drop(['userId', 'timestamp','rating'], axis=1)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
100833,193585,Flint (2017),Drama
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [14]:
user_rating = func.movie_rater(df_joined, 3, 'Action')

       movieId                               title  \
56780     3740  Big Trouble in Little China (1986)   

                                genres  userId  rating   timestamp  
56780  Action|Adventure|Comedy|Fantasy     380     5.0  1494804237  
How do you rate this movie on a scale of 1-5, press n if you have not seen :
1
       movieId        title         genres  userId  rating   timestamp
31083     1387  Jaws (1975)  Action|Horror     330     4.0  1285905302
How do you rate this movie on a scale of 1-5, press n if you have not seen :
1
       movieId             title         genres  userId  rating  timestamp
18379      737  Barb Wire (1996)  Action|Sci-Fi     149     2.0  902085168
How do you rate this movie on a scale of 1-5, press n if you have not seen :
1


In [15]:
user_rating

[{'userId': 1000, 'movieId': 3740, 'rating': '1'},
 {'userId': 1000, 'movieId': 1387, 'rating': '1'},
 {'userId': 1000, 'movieId': 737, 'rating': '1'}]

In [16]:
new_ratings_df = df_new.append(user_rating,ignore_index=True)
new_data = Dataset.load_from_df(new_ratings_df,reader)

In [17]:
svd_ = SVD(n_factors= 50, reg_all=0.05)
svd_.fit(new_data.build_full_trainset())

In [18]:
list_of_movies = []
for m_id in df_new['movieId'].unique():
    list_of_movies.append( (m_id,svd_.predict(1000,m_id)[3]))

In [19]:
list_of_movies

[(1, 4.236438159462414),
 (2, 3.586507344330952),
 (3, 2.51670287745439),
 (4, 3.303356441628164),
 (5, 3.4298436403080794),
 (6, 3.5987885917079163),
 (7, 3.1013442331100163),
 (8, 3.494885878039234),
 (9, 3.204518632667258),
 (10, 3.193893373723333),
 (11, 3.7863486740584187),
 (12, 4.399538198770724),
 (13, 3.6315053158821495),
 (14, 3.427984328134157),
 (15, 3.1638723536373794),
 (16, 3.2536551523250075),
 (17, 3.722055089186057),
 (18, 3.566082613304169),
 (19, 2.7140127372490515),
 (20, 3.5099109999330267),
 (21, 3.119718149067769),
 (22, 2.190526308579413),
 (23, 3.271591898250109),
 (24, 3.401241850968803),
 (25, 4.277224165128358),
 (26, 3.1973829444573627),
 (27, 3.403242655056619),
 (28, 2.9067062070115157),
 (29, 3.7842905212365645),
 (30, 4.1895927121013905),
 (31, 3.7597122977264332),
 (32, 3.664937942765567),
 (34, 3.4049664024039323),
 (36, 2.620269808525286),
 (38, 3.219145867917714),
 (39, 3.638702158136358),
 (40, 3.7853832881742244),
 (41, 2.9805668216371957),
 (42,

In [20]:
ranked_movies = sorted(list_of_movies, key=lambda x:x[1], reverse=True)

In [21]:
ranked_movies

[(53, 4.704026215385585),
 (43, 4.615048992185984),
 (452, 4.434759542045222),
 (276, 4.409155741387929),
 (12, 4.399538198770724),
 (523, 4.35621193419645),
 (171, 4.302162889852852),
 (93, 4.301078206323303),
 (543, 4.295779014477515),
 (371, 4.279018329698782),
 (25, 4.277224165128358),
 (515, 4.273769987832473),
 (337, 4.272369339714336),
 (1, 4.236438159462414),
 (122, 4.232966147363633),
 (475, 4.217674502945716),
 (586, 4.211980515674113),
 (544, 4.211266616976618),
 (348, 4.2109854047455215),
 (243, 4.203321183434554),
 (30, 4.1895927121013905),
 (441, 4.174504156409308),
 (251, 4.166965226993536),
 (169, 4.162601068910145),
 (538, 4.153397558051113),
 (389, 4.148485348454726),
 (52, 4.146596183434162),
 (105, 4.141805139690438),
 (106, 4.140061096481569),
 (413, 4.117359591932155),
 (188, 4.107880653476155),
 (250, 4.090803537223636),
 (429, 4.086252504419232),
 (97, 4.085813029654346),
 (69, 4.082566408051792),
 (154, 4.080288534270002),
 (492, 4.076777410543067),
 (80, 4.073

In [25]:
func.recommended_movies(ranked_movies,movies,4)

Recommendation #  1 :  48    Lamerica (1994)
Name: title, dtype: object 

Recommendation #  2 :  39    Restoration (1995)
Name: title, dtype: object 

Recommendation #  3 :  393    Widows' Peak (1994)
Name: title, dtype: object 

Recommendation #  4 :  238    Milk Money (1994)
Name: title, dtype: object 

